# Identify Songs by Lyrics

This notebook combines notebooks 3 and 4 into one workflow.

## Imports

In [ ]:
import os
import re
import subprocess
import joblib
from pathlib import Path
from datetime import date, datetime, timedelta

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import librosa
import cv2
import easyocr
from rapidfuzz import process, fuzz

from googleapiclient.discovery import build

## CONFIG

Initialise path for .env file (located in the parent directory)

In [ ]:
from dotenv import load_dotenv

env_path = Path("..") / ".env"
load_dotenv(dotenv_path=env_path)

Data Directories (copied from notebook 01)

In [ ]:
DATA_DIR = Path("..") / "data"  # '..' moves up one level to project root

RAW_DATA_DIR = DATA_DIR / "raw"
CLIPS_DATA_DIR = DATA_DIR / "clips"
RESULTS_DIR = DATA_DIR / "results"

STAGING_DIR = CLIPS_DATA_DIR / "segments"
MUSIC_CLIPS_DIR = CLIPS_DATA_DIR / "music"
NOT_MUSIC_CLIPS_DIR = CLIPS_DATA_DIR / "not-music"

# === Create the folders if they don't exist ===
RAW_DATA_DIR.mkdir(parents=True, exist_ok=True)
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
STAGING_DIR.mkdir(parents=True, exist_ok=True)
MUSIC_CLIPS_DIR.mkdir(parents=True, exist_ok=True)
NOT_MUSIC_CLIPS_DIR.mkdir(parents=True, exist_ok=True)

Model directory and filename (copied from notebook 02)

In [ ]:
MODELS_DIR = Path("..") / "models"  # '..' moves up one level to project root
MODEL_FILENAME = "music_classifier.pkl"

# === Create the folders if they don't exist ===
MODELS_DIR.mkdir(parents=True, exist_ok=True)

Output Audio filepath

In [ ]:
OUTPUT_AUDIO = RAW_DATA_DIR / "temp_audio.m4a"

Output Video filepath

In [ ]:
OUTPUT_VIDEO = RAW_DATA_DIR / "temp_video.mp4"

Clip size

In [ ]:
CLIP_SIZE = 5  # Size of analysis window (seconds) - use same value as in notebook 01

YouTube settings

In [ ]:
YOUTUBE_API_KEY = os.getenv("YOUTUBE_API_KEY")
YOUTUBE_BASE_URL = "https://www.youtube.com/watch?v="

NEWLAND_CHANNEL_ID = "UCQ0k5yqN9gikrGiCG8Y3UBQ"
RIVERSIDE_CHANNEL_ID = "UCS_Q5G3O5fydsvRc-uoOanQ"
HESSLE_CHANNEL_ID = "UCl4soOZzigl4RwGWplFmvOw"
ORCHARD_PARK_CHANNEL_ID = "UCIARZq9myh9SrNneKRx5GtA"

Ignore specified warnings

In [ ]:
import warnings

warnings.filterwarnings("ignore", message=".*pin_memory.*")

## SETTINGS

YouTube Search Settings

In [ ]:
youtube_channel_id = NEWLAND_CHANNEL_ID  # Search channel
youtube_q = "Service"  # Search query
filter_keywords_in_title = ["Newland", "Morning"]

# Recommend time window < 3 months to ensure no videos are missed
published_after = "2023-12-17T00:00:00Z"
published_before = "2023-12-19T00:00:00Z"  

Variables for inserting into database

In [ ]:
church_activity_id = 1  # 1: Newland, 2: Network  (check df_activities for others)
link_title_PREFIX = "Newland AM"

Prevent costly song identification if existing links are found for same activity within close proximity to date. --- **WARNING:** can skip songs if service is split into two or more videos

In [ ]:
SKIP_IF_LINKS_MAY_EXIST = False

## Get Data From Database

Load Database URL and Create Database Engine

In [ ]:
from sqlalchemy import create_engine

DB_URL = os.getenv("DB_URL")
engine = create_engine(DB_URL)

### Church Activities

In [ ]:
query = "SELECT * FROM church_activities;"

df_activities = pd.read_sql_query(query, engine)
df_activities

### Lyrics

Query database to get df_lyrics

In [ ]:
query = """
SELECT song_lyrics.song_id, songs.first_line, song_lyrics.content
FROM song_lyrics
JOIN songs ON song_lyrics.song_id = songs.id;
"""

df_lyrics = pd.read_sql_query(query, engine)
df_lyrics.head(3)

Filter out lyric anomalies

In [ ]:
mask = df_lyrics["content"].str.len() > 20
df_lyrics = df_lyrics[mask]
df_lyrics = df_lyrics.reset_index(drop=True)  # Important for iloc and RapidFuzz matches

Add column for cleaned lyrics

In [ ]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()


df_lyrics["cleaned"] = df_lyrics["content"].apply(clean_text)
df_lyrics.head(3)

### Song Usages

Query database to get df_usages

In [ ]:
query = """
SELECT 
    su.id, 
    su.song_id, 
    su.used_date, 
    su.church_activity_id,
    STRING_AGG(syl.id || ':' || syl.start_seconds, ',') as existing_links
FROM song_usage su
LEFT JOIN song_youtube_links syl ON su.id = syl.song_usage_id
GROUP BY su.id, su.song_id, su.used_date, su.church_activity_id
"""
df_usages = pd.read_sql_query(query, engine)

# Convert "id:time,id:time" into a list of dicts: [{'id': 1, 't': 300}, ...]
def parse_links(x):
    if pd.isna(x): return []
    return [{"id": int(i.split(':')[0]), "t": int(i.split(':')[1])} for i in x.split(',')]

df_usages['existing_links'] = df_usages['existing_links'].apply(parse_links)
df_usages.head(3)

Create index for fast lookup

In [ ]:
df_usages_lookup = df_usages.set_index(['song_id', 'used_date', 'church_activity_id']).sort_index()

## Get YouTube Videos (uses API)

Query YouTube Videos using API

In [ ]:
youtube = build('youtube', 'v3', developerKey=YOUTUBE_API_KEY)

request = youtube.search().list(
    part="snippet",
    q=youtube_q,
    channelId=youtube_channel_id,
    type="video",
    publishedAfter=published_after,
    publishedBefore=published_before,
    maxResults=50,
    order="date",
)
response = request.execute()

# Sort the items list in-place by the 'publishedAt' string
# Since ISO dates (YYYY-MM-DD) sort correctly alphabetically, this works fine
response['items'].sort(key=lambda x: x['snippet']['publishedAt'])

results_count = len(response['items'])
print(results_count)
if results_count >= 50:
    print("❗WARNING: reached maximum results count - some videos may have been missed!")

Build youtube_videos (list of dictionaries)

In [ ]:
youtube_videos = [
    {
        "url": f"{YOUTUBE_BASE_URL}{item["id"]["videoId"]}",
        "title": item["snippet"]["title"],
        "date": datetime.fromisoformat(item["snippet"]["publishedAt"].replace("Z", "+00:00")).date(),
        "church_activity_id": church_activity_id,
    } for item in response["items"]
    ]
print(f"{len(youtube_videos)} videos match search")

Filter youtube_videos by keyword in title

In [ ]:
youtube_videos = [
    v for v in youtube_videos 
    if any(kw.lower() in v["title"].lower() for kw in filter_keywords_in_title)
]
print(f"{len(youtube_videos)} videos match keyword filters")

## Initialize models

In [ ]:
model = joblib.load(MODELS_DIR / MODEL_FILENAME)
reader = easyocr.Reader(['en'])

## Helper Functions

Generic helpers

In [ ]:
def print_status(message, width=80):
    print(message.ljust(width), end="\r")


def _get_dates_to_try(target_date):
    # YouTube video date might be late by up to 2 days
    return [target_date, target_date - timedelta(days=1), target_date - timedelta(days=2)]


Download Helpers

In [ ]:
def download_audio(output_path: Path, youtube_url: str):
    print_status("Downloading audio...")
    
    command = [
        "yt-dlp",
        "-q",
        "--force-overwrites",
        "-f", "wa[ext=m4a]/wa/ba",
        "--extract-audio",
        "--audio-format", "m4a",
        "-o", str(output_path),
        youtube_url,
    ]

    subprocess.run(command, check=True)

    return output_path


def download_video(output_path: Path, youtube_url: str):
    print_status("Downloading video...")
    
    command = [
        "yt-dlp",
        "-q",
        "--force-overwrites",
        "-f", "bv*[height<=480][ext=mp4]/wv*[ext=mp4]",
        "-o", str(output_path),
        youtube_url,
    ]

    subprocess.run(command, check=True)

    return output_path


Audio Helpers

In [ ]:
def plot_music_blocks(df, save_plot_path=None):

    df_plot = df.copy()

    # 1. Prepare data for plotting
    # Convert labels to numbers (1 for music, 0 for not-music)
    df_plot["label_num"] = df_plot["label"].map({"music": 1, "not-music": 0})

    # 2. Setup the plot
    plt.figure(figsize=(15, 4))
    plt.step(
        df_plot["start_sec"] / 60,
        df_plot["label_num"],
        where="post",
        color="teal",
        linewidth=2,
    )

    # 3. Formatting
    plt.fill_between(
        df_plot["start_sec"] / 60,
        df_plot["label_num"],
        step="post",
        alpha=0.3,
        color="teal",
    )
    plt.yticks([0, 1], ["Not-Music", "Music"])
    plt.xlabel("Time (Minutes)")
    plt.ylabel("Classification")
    plt.title("Music Detection Timeline")
    plt.grid(axis="x", linestyle="--", alpha=0.7)

    plt.tight_layout()

    if save_plot_path:
        plt.savefig(save_plot_path, dpi=300, bbox_inches="tight")
        print(f"Plot saved to: {save_plot_path}")

    plt.show()


def generate_music_timeline(file_path, window_sec, model, sr=22050):
    total_duration = _get_duration(file_path)
    total_chunks = int(total_duration // window_sec)

    # Calculate bytes per chunk: (seconds * rate * 4 bytes for float32)
    bytes_per_chunk = window_sec * sr * 4
    results = []

    # Use FFmpeg to pipe RAW PCM data to Python
    command = [
        "ffmpeg",
        "-i",
        file_path,
        "-f",
        "f32le",
        "-acodec",
        "pcm_f32le",
        "-ar",
        str(sr),
        "-ac",
        "1",
        "-",
    ]

    process = subprocess.Popen(
        command, stdout=subprocess.PIPE, stderr=subprocess.DEVNULL
    )

    chunk_idx = 0
    while True:
        raw_bytes = process.stdout.read(bytes_per_chunk)
        if not raw_bytes or len(raw_bytes) < bytes_per_chunk:
            break

        # Convert bytes to numpy
        y_block = np.frombuffer(raw_bytes, dtype=np.float32)

        # --- FEATURE EXTRACTION ---
        mfccs = librosa.feature.mfcc(y=y_block, sr=sr, n_mfcc=13)
        mfccs_mean = np.mean(mfccs.T, axis=0)
        mfccs_std = np.std(mfccs.T, axis=0)
        centroid = librosa.feature.spectral_centroid(y=y_block, sr=sr)
        centroid_mean = np.mean(centroid)

        features = np.hstack([mfccs_mean, mfccs_std, centroid_mean]).reshape(1, -1)

        # --- PREDICTION ---
        pred_idx = model.predict(features)[0]
        prob = np.max(model.predict_proba(features))

        start_time = chunk_idx * window_sec
        results.append(
            {
                "start_sec": start_time,
                "end_sec": start_time + window_sec,
                "label": "music" if pred_idx == 1 else "not-music",
                "confidence": round(prob, 4),
            }
        )

        chunk_idx += 1

        # --- DISPLAY PERCENTAGE UPDATE ---
        if (chunk_idx % 20 == 0) or (chunk_idx == total_chunks):
            percent = int((chunk_idx / total_chunks) * 100)
            # Limits display to 100% and prints on one line
            print_status(
                f"Generating Music Timeline: {min(100, percent):>3}% complete"
            )

    process.terminate()
    return pd.DataFrame(results)


def _get_duration(file_path):
    cmd = [
        "ffprobe",
        "-v",
        "error",
        "-show_entries",
        "format=duration",
        "-of",
        "default=noprint_wrappers=1:nokey=1",
        file_path,
    ]
    return float(subprocess.check_output(cmd))


def extract_music_blocks(df, min_duration_sec=120, max_gap_seconds=15):
    print_status("Extracting music blocks...")
    
    # Create a copy so we don't overwrite the original dataframe
    df_clean = df.copy()

    # 1. Convert to numeric for processing
    df_clean["is_music"] = df_clean["label"].map({"music": 1, "not-music": 0})

    # 2. SMOOTHING: Median filter - requires at least 2/3 chunks to match.
    df_clean["is_music"] = (
        df_clean["is_music"]
        .rolling(window=3, center=True)
        .median()
        .fillna(df_clean["is_music"])
    )

    # 3. BRIDGE GAPS: check if music exists within the 'max_gap' range
    # Treat "not-music" sections as "music" if shorter than max_gap_seconds,
    gap_chunks = max_gap_seconds // CLIP_SIZE
    df_clean["is_music"] = (
        df_clean["is_music"]
        .rolling(window=gap_chunks, center=True, min_periods=1)
        .max()
    )

    # 4. Identify song blocks using cumsum logic
    df_clean["block_id"] = (
        df_clean["is_music"] != df_clean["is_music"].shift()
    ).cumsum()

    # 5. Group and Aggregate
    blocks = df_clean.groupby("block_id").agg(
        {"is_music": "first", "start_sec": "min", "end_sec": "max"}
    )

    blocks["duration"] = blocks["end_sec"] - blocks["start_sec"]

    # 6. Filter by music label (1) and min_duration threshold
    songs = blocks[
        (blocks["is_music"] == 1) & (blocks["duration"] >= min_duration_sec)
    ].copy()

    # Formatting helper
    def format_time(seconds):
        return f"{int(seconds // 60):02d}:{int(seconds % 60):02d}"

    songs["start_timestamp"] = songs["start_sec"].apply(format_time)
    songs["end_timestamp"] = songs["end_sec"].apply(format_time)

    songs = songs.reset_index()  # moves block_id from index to column

    return songs[
        [
            "block_id",
            "start_timestamp",
            "end_timestamp",
            "start_sec",
            "end_sec",
            "duration",
        ]
    ]


Video Helpers

In [ ]:
def populate_song_blocks(music_blocks, file_path, youtube_url, video=None, df_usages_lookup=None, debug=False):

    print_status("Populating song blocks...")

    # Load video capture
    cap = cv2.VideoCapture(file_path)

    # Define video properties
    # fps = cap.get(cv2.CAP_PROP_FPS)
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    lyric_start_row = int(height * 0.6)
    lyric_end_row   = height

    SHIFT_TIME = 30  # In case lyrics aren't showing at start and end of song

    song_blocks = {}
    for _, row in music_blocks.iterrows():
        block_id = row['block_id']
        block_start_time = row['start_sec']
        block_end_time = row['end_sec']
        
        song_blocks[block_id] = []
        
        song_start_time = block_start_time
        song_start_time_shifted = song_start_time + SHIFT_TIME
        block_end_time_shifted = block_end_time - SHIFT_TIME
        
        if debug:
            print(f"\n--- Processing Music Block: {block_start_time}s to {block_end_time}s ---")

        # Get song_id at end (once)
        song_id_end = None
        while (song_id_end is None) and (block_end_time_shifted > block_start_time):
            song_end = _get_best_matching_song(cap, lyric_start_row, lyric_end_row, width, block_end_time_shifted, video=video, df_usages_lookup=df_usages_lookup, debug=debug)
            if not song_end:
                block_end_time_shifted -= SHIFT_TIME # Shift back if nothing found
                continue
            song_id_end = song_end["id"]
        if song_id_end is None:
            if debug:
                print("NO SONG FOUND IN THIS SONG BLOCK")
            continue
        
        # ===== GET SONG(S) IN SONG BLOCK =====
        while song_start_time_shifted < block_end_time_shifted:

            # Get song_id at start of song
            song_id_start = None
            while (song_id_start is None) and (song_start_time_shifted < block_end_time_shifted):
                song_start = _get_best_matching_song(cap, lyric_start_row, lyric_end_row, width, song_start_time_shifted, video=video, df_usages_lookup=df_usages_lookup, debug=debug)
                if not song_start:
                    song_start_time_shifted += SHIFT_TIME # Shift forward if nothing found
                    continue
                song_id_start = song_start["id"]
                song_first_line_start = song_start["first_line"]
            if song_id_start is None:
                if debug:
                    print("NO SONG FOUND IN THIS SONG BLOCK")
                break

            # Check if song covers rest of song block
            if song_id_start == song_id_end:
                if debug:
                    print(f"SONG COMPLETED: {song_start}")
                
                song_blocks[block_id].append({
                    "id": song_id_start,
                    "first_line": song_first_line_start,
                    "start": song_start_time,
                    "end": block_end_time,
                    "start_format": _format_timestamp(song_start_time),
                    "end_format": _format_timestamp(block_end_time),
                    "url": youtube_url,
                    })
                break # Entire block is one song, we are done with this block

            # ============= Multiple songs in song block =============
            if debug:
                print(f"MULTIPLE SONGS IN BLOCK - commence binary search..")

            # Binary search for the transition point
            end_time = _get_song_end_time(song_start_time_shifted, block_end_time_shifted, lyric_start_row, lyric_end_row, width, song_id_start, cap, video=video, df_usages_lookup=df_usages_lookup, debug=debug)

            # Assume delay in changing lyrics to new song
            end_time -= 5

            if debug:
                print(f"SONG COMPLETED: {song_start}")

            song_blocks[block_id].append({
                "id": song_id_start,
                "first_line": song_first_line_start,
                "start": song_start_time,
                "end": end_time,
                "start_format": _format_timestamp(song_start_time),
                "end_format": _format_timestamp(end_time),
                "url": youtube_url,
                })
            
            song_start_time = end_time  # Move to the start of the next song
            song_start_time_shifted = song_start_time + SHIFT_TIME

    cap.release()
    return song_blocks


def _format_timestamp(seconds: int) -> str:
    h, r = divmod(int(seconds), 3600)
    m, s = divmod(r, 60)
    return f"{h:02d}:{m:02d}:{s:02d}"


def _identify_songs_from_lyrics(search_text, df_lyrics, threshold=80):
    search_text_cleaned = clean_text(search_text)
    
    if not search_text_cleaned or len(search_text_cleaned) < 10:
        return []

    # extract returns a list of (string, score, index) tuples
    results = process.extract(
        search_text_cleaned, 
        df_lyrics['cleaned'], 
        scorer=fuzz.partial_ratio,
        # scorer=fuzz.token_set_ratio,  # could try this instead
        score_cutoff=threshold,
        limit=5 
    )

    matches = []
    for _, score, idx in results:
        match_row = df_lyrics.iloc[idx]
        matches.append({
            "id": int(match_row["song_id"]),
            "first_line": match_row["first_line"],
            "score": round(float(score), 1)
        })

    # RapidFuzz's extract automatically sorts by score DESC
    return matches


def _display_images(lyric_zone, thresh):
    """View original image frame and formatted frame seen by OCR"""
    fig, ax = plt.subplots(1, 2, figsize=(15, 5))

    ax[0].imshow(cv2.cvtColor(lyric_zone, cv2.COLOR_BGR2RGB))
    ax[0].set_title("Original Crop")

    ax[1].imshow(thresh, cmap='gray')
    ax[1].set_title("Thresholded (OCR Input)")

    plt.show()


def _get_screen_text(frame, lyric_start_row, lyric_end_row, width, debug):
    """Return text shown on specified region of screen"""
    
    # 1. CROP: Keep only the bottom section
    lyric_zone = frame[lyric_start_row:lyric_end_row, 0:width]

    # 2. GRAYSCALE: Process only the small cropped area
    gray_lyric = cv2.cvtColor(lyric_zone, cv2.COLOR_BGR2GRAY)

    # 3. THRESHOLD: High Contrast (Makes OCR 2x more accurate)
    _, thresh = cv2.threshold(gray_lyric, 200, 255, cv2.THRESH_BINARY)
    
    # 4. OCR: read lyrics
    results = reader.readtext(thresh, detail=0)
    if not results:
        if debug:
            _display_images(lyric_zone, thresh)
        return None

    # 5. FORMAT: convert from list to string
    text = " ".join(results)    
    if len(text) < 20:  # catch random noise or too few lyrics
        return None
    
    return text


def _get_song_end_time(left, right, lyric_start_row, lyric_end_row, width, song_id, cap, debug, video=None, df_usages_lookup=None):
    """Binary search for time when changes from one song to another"""

    while (right - left) > 2:  # Stop when within 2 seconds
        mid = (left + right) // 2
        
        song = _get_best_matching_song(cap, lyric_start_row, lyric_end_row, width, mid, video=video, df_usages_lookup=df_usages_lookup, debug=debug)
        if not song:
            # Catch no song match (e.g. lyrics not displayed on screen)
            right = right - 10
            continue
        
        mid_id = song["id"]

        if mid_id == song_id:
            left = mid
        else:
            right = mid

    return left


def _get_best_matching_song(cap, lyric_start_row, lyric_end_row, width, sec, debug, video=None, df_usages_lookup=None):
    cap.set(cv2.CAP_PROP_POS_MSEC, sec * 1000)
    ret, frame = cap.read()
    if not ret:
        return None

    text = _get_screen_text(frame, lyric_start_row, lyric_end_row, width, debug=debug)
    songs = _identify_songs_from_lyrics(text, df_lyrics)
    
    if not songs:
        if debug:
            print(f"{sec}s: NO MATCH FOUND: text={text}")
        return None
    
    if len(songs) == 1 or video is None:
        # If video is None, can't search usage data anyway since don't have video date
        return songs[0]

    # Search usage data 
    if True:
        song_candidates = [s["first_line"] for s in songs]
        print(f"Song candidates: {', '.join(song_candidates)}")
    target_date = video["date"]
    church_activity_id = video["church_activity_id"]
    for song in songs:
        # Loop until match is found
        for date_candidate in _get_dates_to_try(target_date):
            try:
                match = df_usages_lookup.loc[(song["id"], date_candidate, church_activity_id)]
                if not match.empty:
                    return song
            except KeyError:
                continue
    
    # Fallback
    return songs[0]


def display_song_block_summary(song_blocks):
    for block_id in song_blocks.keys():
        print("---".ljust(80))
        for song in song_blocks[block_id]:
            print(f'{song["start_format"]} - {song["url"]}&t={song["start"]} - {song["first_line"]}')


SongYouTubeLink Helpers

In [ ]:
from sqlalchemy import text
from sqlalchemy.exc import SQLAlchemyError


def add_song_youtube_link(
    df_usages_lookup,
    video,
    song,
    debug=False,
):

    target_date = video["date"]
    church_activity_id = video["church_activity_id"]

    match = None
    final_used_date = None

    # Loop until match is found
    for date_candidate in _get_dates_to_try(target_date):
        try:
            match = df_usages_lookup.loc[(song["id"], date_candidate, church_activity_id)]
            final_used_date = date_candidate
            break # Exit loop if found
        except KeyError:
            continue

    # Handle case when usage not found
    if match is None:
        print_link_update_status_msg("❗ No usage found", song, final_used_date, None, None)
        return False

    # Extract Usage ID
    usage_id = int(match["id"].iloc[0] if hasattr(match, "iloc") else match["id"])

    # Perform Duplicate Check
    existing_id = _get_existing_link_id_at_time(match, song["start"])
    if existing_id:
        if debug:
            print_link_update_status_msg("ℹ️ SKIP - Timestamp match", song, final_used_date, usage_id, existing_id)
        return existing_id

    # Build video title
    title = _build_video_title(link_title_PREFIX, final_used_date, song["first_line"])

    # Define query
    insert_query = text(
        """
        INSERT INTO song_youtube_links 
        (song_usage_id, url, start_seconds, end_seconds, title, is_featured)
        VALUES (:song_usage_id, :url, :start_seconds, :end_seconds, :title, False)
        RETURNING id
        """
    )
    params = {
        "song_usage_id": usage_id,
        "url": song["url"],
        "start_seconds": song["start"],
        "end_seconds": song["end"],
        "title": title,
    }

    # Add entry to SongYouTubeLink
    new_link_id = None
    try:
        with engine.begin() as conn:
            result = conn.execute(insert_query, params)
            new_link_id = result.scalar() 
    except SQLAlchemyError as e:
        print_link_update_status_msg(f"❗ Database error: {e}", song, final_used_date, None, None)
        return None
    
    # === UPDATE df_usages_lookup ===
    # 1. Target the specific index
    idx = (song["id"], final_used_date, church_activity_id)

    # 2. Get the current list (handling potential multiple rows)
    matches = df_usages_lookup.loc[idx, "existing_links"]
    current_val = matches.iloc[0] if hasattr(matches, "iloc") else matches

    # 3. Ensure it's a list and append
    if not isinstance(current_val, list):
        current_val = []
    current_val.append({"id": new_link_id, "t": song["start"]})

    # 4. WRITE BACK using .loc (this safely updates all duplicate rows at once)
    # Note: We wrap in a Series or just use .loc directly to avoid 'Must be same length' errors
    df_usages_lookup.loc[idx, "existing_links"] = [current_val] * (len(matches) if hasattr(matches, "__len__") else 1)

    # Log success
    if debug:
        print_link_update_status_msg(f"✅ Added link", song, final_used_date, usage_id, new_link_id)

    return new_link_id


def _build_video_title(link_title_PREFIX, final_used_date, song_first_line):
    return f"{link_title_PREFIX} {final_used_date.strftime("%d/%m/%y")} - {song_first_line}"


def _get_existing_link_id_at_time(match, current_start, buffer=15):
    # Get the list of dictionaries
    links = match["existing_links"].iloc[0] if hasattr(match, "iloc") else match["existing_links"]
    
    if not links:
        return None
        
    for link in links:
        # Check if the current timestamp is within the buffer of an existing record
        if abs(link["t"] - current_start) <= buffer:
            return link["id"] # Return the specific link_id
            
    return None


def print_link_update_status_msg(headline, song, used_date, usage_id, link_id):
    print(f'{headline} - {song["url"]}&t={song["start"]}')
    print(f' - {used_date} - usage_id {usage_id} - link_id {link_id} - song_id {song["id"]} - {song["first_line"]}')


def usage_link_may_already_exist(df_usages_lookup, target_date, church_activity_id):
    """
    Used for quick search to avoid costly song identification.
    
    Since it doesn't check for specific songs, this won't catch situations
    where some song usage links have been added but others have not.
    """
    for date_candidate in _get_dates_to_try(target_date):
        try:
            # with .xs can skip 'song_id' and filter by the other levels
            match = df_usages_lookup.xs((date_candidate, church_activity_id), 
                                        level=('used_date', 'church_activity_id'))
            if (match["link_count"] > 0).any():
                return True
        except KeyError:
            continue
    return False


SongYouTubeLink Manual Helpers

In [ ]:
def get_song_usage_id(song_id, used_at_activity_id, used_date):
    # Convert string to date object
    target_date_obj = date.fromisoformat(used_date)

    # Find song usages which match criteria
    match = df_usages[
        (df_usages["church_activity_id"] == used_at_activity_id) &
        (df_usages["song_id"] == song_id) &
        (df_usages["used_date"] == target_date_obj)
    ]
    if match.empty:
        print("❗ No song usages match criteria")
        return None
    
    if len(match) > 1:
        print("❗ Multiple song usages match criteria! Please specify song_usage_id manually")
        display(match)
        return None
    
    song_usage_id = int(match["id"].iloc[0])
    print(f"✅ song_usage_id set to {song_usage_id}")
    
    return song_usage_id


def convert_time_str_to_seconds(time_str):
    return sum(int(x) * 60**i for i, x in enumerate(reversed(time_str.split(":"))))


def add_song_link_manually(
    song_usage_id,
    url,
    start_seconds,
    end_seconds,
    title,
    is_featured=False,
    description=None,
    thumbnail_key=None,
):
    insert_query = text(
        """
        INSERT INTO song_youtube_links 
        (song_usage_id, url, start_seconds, end_seconds, title, is_featured, description, thumbnail_key)
        VALUES (:song_usage_id, :url, :start_seconds, :end_seconds, :title, :is_featured, :description, :thumbnail_key)
        RETURNING id
        """
    )
    params = {
        "song_usage_id": song_usage_id,
        "url": url,
        "start_seconds": start_seconds,
        "end_seconds": end_seconds,
        "title": title,
        "is_featured": is_featured,
        "description": description,
        "thumbnail_key": thumbnail_key,
    }

    # Add entry to SongYouTubeLink
    new_link_id = None
    try:
        with engine.begin() as conn:
            result = conn.execute(insert_query, params)
            new_link_id = result.scalar()
            print(f"✅ Added link {new_link_id}")
    except SQLAlchemyError as e:
        print(f"❗ Database error: {e}")
    
    return new_link_id


## MANUAL_MODE

This section allows for manually adding song links if automated procedure doesn't work.

Use with care as duplicate checks are disabled.

In [ ]:
MANUAL_MODE = False

View activity_id

In [ ]:
if MANUAL_MODE:
    display(df_activities)

View song_id

In [ ]:
song_lyrics_contain = "this earth belongs"

if MANUAL_MODE:
    results = df_lyrics[df_lyrics["content"].str.contains(song_lyrics_contain, case=False, na=False)]
    display(results)

View church_activity_id

In [ ]:
if MANUAL_MODE:
    display(df_activities)

View song usages near date range

In [ ]:
if MANUAL_MODE:

    # --- Required parameters
    url = "https://www.youtube.com/watch?v=Pduvp1q4u4M"
    start_seconds = convert_time_str_to_seconds("18:37")
    end_seconds = convert_time_str_to_seconds("21:09")
    title = "Newland AM 02/04/2023 - This Earth Belongs To God"

    song_id = 1442
    used_at_activity_id = 1
    used_date = "2023-04-02"
    song_usage_id = get_song_usage_id(song_id, used_at_activity_id, used_date)

    if song_usage_id is None:
        raise Exception("song_usage_id not found")

    # --- Optional parameters
    is_featured = False
    description = None
    thumbnail_key = None

    # ============================== Add Song Link ==============================
    new_link_id = add_song_link_manually(
        song_usage_id=song_usage_id,
        url=url,
        start_seconds=start_seconds,
        end_seconds=end_seconds,
        title=title,
        is_featured=False,
        description=None,
        thumbnail_key=None,
    )


## Run Main Procedure

In [ ]:
total_videos = len(youtube_videos)

for i, video in enumerate(youtube_videos, 1):
    
    print(f"\n##### Video {i} of {total_videos} - {video["title"]} ###")

    some_links_found = usage_link_may_already_exist(
        df_usages_lookup, video["date"], video["church_activity_id"])
    
    if some_links_found and SKIP_IF_LINKS_MAY_EXIST:
        print(f'ℹ️ Existing links found for church_activity_id {video["church_activity_id"]} near this date. Skipping.')
        continue

    try:
        # ===== AUDIO =====
        tmp_audio = download_audio(output_path=OUTPUT_AUDIO, youtube_url=video["url"])
        df_blocks = generate_music_timeline(file_path=tmp_audio, window_sec=CLIP_SIZE, model=model)
        # plot_music_blocks(df=df_blocks)
        music_blocks = extract_music_blocks(df_blocks, min_duration_sec=120, max_gap_seconds=15)

        # ===== VIDEO =====
        tmp_video = download_video(output_path=OUTPUT_VIDEO, youtube_url=video["url"])
        song_blocks = populate_song_blocks(file_path=tmp_video, music_blocks=music_blocks, youtube_url=video["url"], video=video, df_usages_lookup=df_usages_lookup)
        # display_song_block_summary(song_blocks)
        
        # Add Song YouTube Links to Database
        for block_id, songs in song_blocks.items():
            for song in songs:
                add_song_youtube_link(
                    df_usages_lookup,
                    video,
                    song,
                    debug=True,
                )
    except subprocess.CalledProcessError as e:
        print(f"❌ Skipping {video['title']} - YouTube Download Failed - {e}")
        continue
    
    except Exception as e:
        # This is the "Safety Net" for anything else
        print(f"❗ Unexpected error processing {video['title']}: {type(e).__name__} - {e}")
        continue 